### Retrieval Chain

In [ ]:
! ollama pull nomic-embed-text

In [ ]:
# Step 1: Load, Chunk, Embed, Store

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

# Load documents from a URL
docs = WebBaseLoader("https://docs.smith.langchain.com/user_guide").load()  

# Split documents into chunks
chunk = RecursiveCharacterTextSplitter()
documents = chunk.split_documents(docs)              
document_texts = [doc.page_content for doc in documents]  # Extract text

# Embed document texts
embed = OllamaEmbeddings(model="nomic-embed-text")
embeddings = embed.embed_documents(document_texts)  # Generate embeddings

# Create and save FAISS vector store
vector = FAISS.from_embeddings(list(zip(document_texts, embeddings)), embedding=embed)
vector.save_local("C:/Users/admin/OneDrive/Desktop/RAG'S/vector db/Faiss-index/")  # Store locally


In [ ]:
import os
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_groq import ChatGroq

# Initialize Embeddings and Load FAISS Vector Store
embed = OllamaEmbeddings(model="nomic-embed-text")
vector = FAISS.load_local("C:/Users/admin/OneDrive/Desktop/RAG'S/vector db/Faiss-index/", embeddings=embed, allow_dangerous_deserialization=True)

# Define the Context and Prompt Template
context = """LangSmith is a platform for developing, monitoring, and testing Large Language Model (LLM) applications."""
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>
Question: {input}""")

# Initialize LLM (Mixtral)
os.environ["GROQ_API_KEY"] = "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"
llm_mixtral = ChatGroq(model="mixtral-8x7b-32768")

# Create Retrieval Chain (Combining Document Chain and Retriever)
document_chain = create_stuff_documents_chain(llm_mixtral, prompt)
retrieval_chain = create_retrieval_chain(vector.as_retriever(), document_chain)

In [14]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?Explain in short."})
print(response["answer"])

LangSmith is a platform that can help with testing LLM (Language Learning Model) applications in several ways. First, it allows developers to create and use datasets as test cases, which can be uploaded in bulk or created on the fly. These test cases can be used to run custom evaluations and score test results. Additionally, LangSmith provides a comparison view for test runs, which allows users to view results for different configurations side-by-side and diagnose regressions in test scores across multiple revisions of an application. LangSmith also allows developers to attach feedback scores to logged traces, filter on traces with specific feedback tags and scores, and send runs to annotation queues for closer inspection and annotation by annotators. These features can help developers identify and root-cause issues, track regressions and improvements, and refine and improve the performance of their LLM applications.
